# Tutorial: Embedding Metadata for Improved Retrieval


- **Level**: Intermediate
- **Time to complete**: 10 minutes
- **Components Used**: [`InMemoryDocumentStore`](https://docs.haystack.deepset.ai/v2.0/docs/inmemorydocumentstore), [`InMemoryEmbeddingRetriever`](https://docs.haystack.deepset.ai/v2.0/docs/inmemoryembeddingretriever), [`SentenceTransformersDocumentEmbedder`](https://docs.haystack.deepset.ai/v2.0/docs/sentencetransformersdocumentembedder), [`SentenceTransformersTextEmbedder`](https://docs.haystack.deepset.ai/v2.0/docs/sentencetransformerstextembedder)
- **Goal**: After completing this tutorial, you'll have learned how to embed metadata information while indexing documents, to improve retrieval.

> This tutorial uses Haystack 2.0 Beta. To learn more, read the [Haystack 2.0 Beta announcement](https://haystack.deepset.ai/blog/introducing-haystack-2-beta-and-advent) or visit the [Haystack 2.0 Documentation](https://docs.haystack.deepset.ai/v2.0/docs).

> ⚠️ Note of caution: The method showcased in this tutorial is not always the right approach for all types of matadata. This method works best when the embedded metadata is meaningful. For example, here we're showcasing embedding the "title" meta field, which can also provide good context for the embedding model.

## Overview

While indexing documents into a document store, we have 2 options: embed the text for that document or embed the text alongside some meaningful metadata. In some cases, embedding meaningful metadata alongside the contents of a document may improve retrieval down the line. 

In this tutorial, we will see how we can embed metadata as well as the text of a document. We will fetch various pages from Wikipedia and index them into an `InMemoryDocumentStore` with metadata information that includes their title, and URL. Next, we will see how retrieval with and without this metadata.

## Setup
### Prepare the Colab Environment

- [Enable GPU Runtime in Colab](https://docs.haystack.deepset.ai/v2.0/docs/enabling-gpu-acceleration)
- [Set logging level to INFO](https://docs.haystack.deepset.ai/v2.0/docs/setting-the-log-level)

### Install Haystack

Install Haystack 2.0 Beta and other required packages with `pip`:

In [1]:
%%bash

pip install haystack-ai wikipedia sentence-transformers

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=17926b00d77f1d294e927f20b0e52e7a137fcd6b219ca85e63570f3b5c7d58f3
  Stored in directory: /Users/tuanacelik/Library/Caches/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


### Enable Telemetry

Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/v2.0/docs/telemetry) for more details.

In [ ]:
from haystack.telemetry import tutorial_running

tutorial_running(39)

## Indexing Documents with Metadata

Create a pipeline to store the small example dataset in the [InMemoryDocumentStore](https://docs.haystack.deepset.ai/v2.0/docs/inmemorydocumentstore) with their embeddings. We will use [SentenceTransformersDocumentEmbedder](https://docs.haystack.deepset.ai/v2.0/docs/sentencetransformersdocumentembedder) to generate embeddings for your Documents and write them to the document store with the [DocumentWriter](https://docs.haystack.deepset.ai/v2.0/docs/documentwriter).

After adding these components to your pipeline, connect them and run the pipeline.

> 💡 The `InMemoryDocumentStore` is the simplest document store to run tutorials with and comes with no additional requirements. This can be changed to any of the other available document stores such as **Weaviate, AstraDB, Qdrant, Pinecone and more**. Check out the [full list of document stores](https://haystack.deepset.ai/integrations?type=Document+Store) with instructions on how to run them. 

First, we'll create a helper function that can create indexing pipelines. We will optionally provide this function with `meta_fields_to_embed`. If provided, the `SentenceTransformersDocumentEmbedder` will be initizlied with metadata to embed alongside the contentes of the document.

For example, the embedder below will be embedding the "url" field as well as the contents of documents:

```python
from haystack.components.embedders import SentenceTransformersTextEmbedder

embedder = SentenceTransformersDocumentEmbedder(meta_fields_to_embed=["url"])
```

In [16]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy

def create_indexing_pipeline(document_store, metadata_fields_to_embed = None):
   document_cleaner = DocumentCleaner()
   document_splitter = DocumentSplitter(split_by='sentence', split_length=2)
   document_embedder = SentenceTransformersDocumentEmbedder(model="thenlper/gte-large", meta_fields_to_embed=metadata_fields_to_embed, device="cuda")
   document_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE)

   indexing_pipeline = Pipeline()
   indexing_pipeline.add_component("cleaner", document_cleaner)
   indexing_pipeline.add_component("splitter", document_splitter)
   indexing_pipeline.add_component("embedder", document_embedder)
   indexing_pipeline.add_component("writer", document_writer)
   
   indexing_pipeline.connect("cleaner", "splitter")
   indexing_pipeline.connect("splitter", "embedder")
   indexing_pipeline.connect("embedder", "writer")
   
   return indexing_pipeline


Next, we can index our documents from various wikipedia articles. We will create 2 indexing pipelines:

- The `indexing_pipeline`: which indexes only the contents of the documents. We will index these documents into `document_store`.
- The `indexing_with_metadata_pipeline`: which indexes meta fields alongside the contents of the documents. We will index these documents into `document_store_with_embedded_metadata`.

In [17]:
import wikipedia
from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore

some_bands="""The Beatles,The Cure""".split(",")

raw_docs=[]

for title in some_bands:
    page = wikipedia.page(title=title, auto_suggest=False)
    doc = Document(content=page.content, meta={"title": page.title, "url": page.url})
    raw_docs.append(doc)

document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
document_store_with_embedded_metadata = InMemoryDocumentStore(embedding_similarity_function="cosine")

indexing_pipeline = create_indexing_pipeline(document_store=document_store)
indexing_with_metadata_pipeline = create_indexing_pipeline(document_store=document_store_with_embedded_metadata, metadata_fields_to_embed=["title"])

indexing_pipeline.run({"cleaner":{"documents": raw_docs}})
indexing_with_metadata_pipeline.run({"cleaner":{"documents": raw_docs}})

config_sentence_transformers.json: 100%|██████████| 124/124 [00:00<00:00, 360kB/s]
README.md: 100%|██████████| 93.0k/93.0k [00:00<00:00, 970kB/s]
sentence_bert_config.json: 100%|██████████| 52.0/52.0 [00:00<00:00, 156kB/s]
config.json: 100%|██████████| 743/743 [00:00<00:00, 2.25MB/s]
model.safetensors: 100%|██████████| 133M/133M [02:41<00:00, 826kB/s]  
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 1.92MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.09MB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 1.11MB/s]
Batches: 100%|██████████| 2/2 [00:38<00:00, 19.09s/it]


{'writer': {'documents_written': 47}}

## Comparing Retrieval With and Without Embedded Metadata

As a final step, we will be creating a retrieval pipeline that will have 2 retrivers:
- First: retrieving from the `document_store`, where we have not embedded metadata.
- Second: retireving from the `document_store_with_embedded_metadata`, where we have embedded metadata.

We will then be able to compare the restults and see if embedding metadata has helped with retrieval in this case.

> 💡 Here, we are using the `InMemoryEmbeddintRetriever` because we used the `InMemoryDocumentStore` above. If you're using another documet store, change this to use the accompanying embedding retriever for the document store you are using. Check out the [Embedders Documentation](https://docs.haystack.deepset.ai/v2.0/docs/embedders) for a full list

In [19]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder(model="thenlper/gte-large"))
retrieval_pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store, scale_score=False, top_k=3))
retrieval_pipeline.add_component("retriever_with_embeddings", InMemoryEmbeddingRetriever(document_store=document_store_with_embedded_metadata, scale_score=False, top_k=3))

retrieval_pipeline.connect("text_embedder", "retriever")
retrieval_pipeline.connect("text_embedder", "retriever_with_embeddings")


Let's run the pipeline and compare the restuls from `retriever` and `retirever_with_embeddings`:

In [25]:
result = retrieval_pipeline.run({"text_embedder":{"text":"Have the beatles ever been to Bangor?"}})

print("Retriever Results:\n")
for doc in result['retriever']['documents']:
    print(doc)

print("Retriever with Embeddings Results:\n")
for doc in result['retriever_with_embeddings']['documents']:
    print(doc)

Batches: 100%|██████████| 1/1 [00:00<00:00, 16.26it/s]

Retriever Results:

Document(id=d75644994513b31889d1109b0dce0db16d1106f3a9f2b0b8225466d68b6d6036, content: 'The Beatles were an English rock band formed in Liverpool in 1960, comprising John Lennon, Paul McCa...', meta: {'title': 'The Beatles', 'url': 'https://en.wikipedia.org/wiki/The_Beatles', 'source_id': '5a1287040b90a84960d692ab40afd95ea64f198a20648c73b69ae5be301e5234'}, score: 0.7120939620182278)
Document(id=873df318a5704451707432e70755ebe07bc98c03b773025e2bcd8acfd14544c5, content: '2001. McCartney and Starr remain musically active.
The Beatles are the best-selling music act of all...', meta: {'title': 'The Beatles', 'url': 'https://en.wikipedia.org/wiki/The_Beatles', 'source_id': '5a1287040b90a84960d692ab40afd95ea64f198a20648c73b69ae5be301e5234'}, score: 0.6633389259123764)
Document(id=82d24eb9936ec0f54de2ca61fe46c12bf7097eecbb2da58f79371feb03465c7b, content: 'had previously accomplished while touring with artists from the US. A similar situation arose during...', meta: {'title'

## What's next

🎉 Congratulations! You've simplified your pipeline run with a Multiplexer!

If you liked this tutorial, there's more to learn about Haystack 2.0:
- [Creating a Hybrid Retrieval Pipeline](https://haystack.deepset.ai/tutorials/33_hybrid_retrieval)
- [Building Fallbacks to Websearch with Conditional Routing](https://haystack.deepset.ai/tutorials/36_building_fallbacks_with_conditional_routing)
- [Model-Based Evaluation of RAG Pipelines](https://haystack.deepset.ai/tutorials/35_model_based_evaluation_of_rag_pipelines)

To stay up to date on the latest Haystack developments, you can [sign up for our newsletter](https://landing.deepset.ai/haystack-community-updates?utm_campaign=developer-relations&utm_source=tutorial&utm_medium=embedding-metadata) or [join Haystack discord community](https://discord.gg/haystack).

Thanks for reading!